In [36]:
import pandas as pd
import numpy as np
import os

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
dir_path = "/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/"
filelist =[]
for file in os.listdir(dir_path):
    # if file.endswith("_v2.csv"):
    filelist.append(dir_path + file)
filelist

['/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/13.tweets_neg.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/1.gt_bitcoin.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/10.news_senti.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/12.tweets_pos.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/14.twitter_btccrash_pos.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/2.gt_Buy_Bitcoin.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/7.s&p_twitter.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/8.CBDC_uncertainty.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/15.twitter_btccrash_neg.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/6.gt_covid.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Sentiment_Data/5.gt_U

In [ ]:
filelist[0][68:-4]

'13.tweets_neg'

In [39]:
dir_path = "/content/drive/MyDrive/S&P Crypto Project/Final_Data/Crypto_Data/"
coinlist =[]
for file in os.listdir(dir_path):
    # if file.endswith("_v2.csv")and "binance" not in file and "uniswap" not in file and "chainlink" not in file:
        coinlist.append(dir_path + file)
coinlist

['/content/drive/MyDrive/S&P Crypto Project/Final_Data/Crypto_Data/SP_BDM.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Crypto_Data/SP_BDM_exlarge.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Crypto_Data/bitcoin.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Final_Data/Crypto_Data/ethereum.csv']

In [ ]:
dir_path = "/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/"
gtlist =[]
for file in os.listdir(dir_path):
    if file.endswith("_v2.csv"):
        gtlist.append(dir_path + file)
gtlist

['/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/bitcoin_v2.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/Sell_Bitcoin_v2.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/covid_v2.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/Ukraine war_v2.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/Buy_Bitcoin_v2.csv',
 '/content/drive/MyDrive/S&P Crypto Project/Market Sentiment Data/Google Trends/gt v2/ethereum_v2.csv']

In [ ]:
coinlist[0][65:-4]

'SP_BDM'

In [40]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag = 5
test   = 'ssr_chi2test'
def grangers_causation_matrix(title,data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [til + '_x' for til in title]
    df.index = [til + '_y' for til in title]
    display(df)
    # return df

### Return Granger Test

In [41]:
for file in filelist:
  for coin in coinlist:
    title = [file[68:-4],coin[65:-4]]
    df_sentiment = pd.read_csv(file)
    # print(df_sentiment.columns)
    df_sentiment = df_sentiment.iloc[:,[0,1]]
    df_sentiment.columns=['date','index']
    df_sentiment['change'] = df_sentiment['index'].pct_change()
    df_coin = pd.read_csv(coin)
    df_coin = df_coin[['date','price']]
    df_coin['return'] = df_coin['price'].pct_change()
    df_new = df_sentiment.merge(df_coin, left_on = "date", right_on = "date", how="left")
    df_new.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_new.dropna(inplace=True)

    variables = ['return','change']
    grangers_causation_matrix(title, df_new, variables)


,13.tweets_neg_x,SP_BDM_x
13.tweets_neg_y,1.0000,0.198
SP_BDM_y,0.0201,1.000


,13.tweets_neg_x,SP_BDM_exlarge_x
13.tweets_neg_y,1.0000,0.2241
SP_BDM_exlarge_y,0.0302,1.0000


,13.tweets_neg_x,bitcoin_x
13.tweets_neg_y,1.0000,0.7057
bitcoin_y,0.0799,1.0000


,13.tweets_neg_x,ethereum_x
13.tweets_neg_y,1.0000,0.7681
ethereum_y,0.1835,1.0000


,1.gt_bitcoin_x,SP_BDM_x
1.gt_bitcoin_y,1.0000,0.105
SP_BDM_y,0.1072,1.000


...

,4.gt_ethereum_x,ethereum_x
4.gt_ethereum_y,1.0000,0.0002
ethereum_y,0.3618,1.0000


,11.fear&greed_x,SP_BDM_x
11.fear&greed_y,1.0,0.4754
SP_BDM_y,0.0,1.0000


,11.fear&greed_x,SP_BDM_exlarge_x
11.fear&greed_y,1.0,0.2809
SP_BDM_exlarge_y,0.0,1.0000


,11.fear&greed_x,bitcoin_x
11.fear&greed_y,1.0,0.1911
bitcoin_y,0.0,1.0000


,11.fear&greed_x,ethereum_x
11.fear&greed_y,1.0,0.1352
ethereum_y,0.0,1.0000


In [ ]:
for gt in gtlist:
  for coin in coinlist:
    title = [gt[84:-7]+"_gt",coin[54:-7]]
    df_sentiment = pd.read_csv(gt)
    df_sentiment.columns = ['date','index']
    df_sentiment['change'] = df_sentiment['index'].pct_change()
    df_coin = pd.read_csv(coin)
    df_coin['return'] = df_coin['price'].pct_change()
    df_new = df_sentiment.merge(df_coin, left_on = "date", right_on = "Date", how="left")
    df_new.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_new.dropna(inplace=True)

    variables = ['return','change']
    grangers_causation_matrix(title, df_new, variables)

,bitcoin_gt_x,bitcoin_x
bitcoin_gt_y,1.0000,0.6145
bitcoin_y,0.1214,1.0000


,bitcoin_gt_x,SP_BDM_exlarge_x
bitcoin_gt_y,1.0000,0.0004
SP_BDM_exlarge_y,0.0615,1.0000


,bitcoin_gt_x,SP_BDM_x
bitcoin_gt_y,1.0000,0.1113
SP_BDM_y,0.1083,1.0000


,bitcoin_gt_x,ether_x
bitcoin_gt_y,1.0000,0.1145
ether_y,0.0704,1.0000


,Sell_Bitcoin_gt_x,bitcoin_x
Sell_Bitcoin_gt_y,1.000,0.3907
bitcoin_y,0.215,1.0000


...

,Buy_Bitcoin_gt_x,ether_x
Buy_Bitcoin_gt_y,1.0000,0.0795
ether_y,0.7652,1.0000


,ethereum_gt_x,bitcoin_x
ethereum_gt_y,1.0000,0.0011
bitcoin_y,0.5705,1.0000


,ethereum_gt_x,SP_BDM_exlarge_x
ethereum_gt_y,1.0000,0.0038
SP_BDM_exlarge_y,0.1928,1.0000


,ethereum_gt_x,SP_BDM_x
ethereum_gt_y,1.0000,0.0009
SP_BDM_y,0.0785,1.0000


,ethereum_gt_x,ether_x
ethereum_gt_y,1.000,0.0002
ether_y,0.362,1.0000


### Change in Volatility Granger Test

In [ ]:
for gt in gtlist:
  for coin in coinlist:
    title = [gt[84:-7]+"_gt_rv",coin[54:-7]+"_rv"]
    df_sentiment = pd.read_csv(gt)
    df_sentiment.columns = ['date','index']
    df_sentiment['change'] = df_sentiment['index'].pct_change()
    df_sentiment['sen_vol'] = df_sentiment['change'].rolling(30).std()
    df_sentiment['sen_rv'] = df_sentiment['sen_vol'].pct_change()
    df_coin = pd.read_csv(coin)
    df_coin['return'] = df_coin['price'].pct_change()
    df_coin['coin_vol'] = df_coin['return'].rolling(30).std()
    df_coin['coin_rv'] = df_coin['coin_vol'].pct_change()
    df_new = df_sentiment.merge(df_coin, left_on = "date", right_on = "Date", how="left")
    df_new.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_new.dropna(inplace=True)
    
    variables = ['coin_rv','sen_rv']
    grangers_causation_matrix(title, df_new, variables)

,bitcoin_gt_rv_x,bitcoin_rv_x
bitcoin_gt_rv_y,1.0000,0.0258
bitcoin_rv_y,0.7378,1.0000


,bitcoin_gt_rv_x,SP_BDM_exlarge_rv_x
bitcoin_gt_rv_y,1.0000,0.1529
SP_BDM_exlarge_rv_y,0.7848,1.0000


,bitcoin_gt_rv_x,SP_BDM_rv_x
bitcoin_gt_rv_y,1.000,0.0634
SP_BDM_rv_y,0.883,1.0000


,bitcoin_gt_rv_x,ether_rv_x
bitcoin_gt_rv_y,1.0000,0.0132
ether_rv_y,0.7754,1.0000


,Sell_Bitcoin_gt_rv_x,bitcoin_rv_x
Sell_Bitcoin_gt_rv_y,1.0000,0.2078
bitcoin_rv_y,0.2308,1.0000


...

,Buy_Bitcoin_gt_rv_x,ether_rv_x
Buy_Bitcoin_gt_rv_y,1.0000,0.0285
ether_rv_y,0.5368,1.0000


,ethereum_gt_rv_x,bitcoin_rv_x
ethereum_gt_rv_y,1.0000,0.6126
bitcoin_rv_y,0.0002,1.0000


,ethereum_gt_rv_x,SP_BDM_exlarge_rv_x
ethereum_gt_rv_y,1.0000,0.6497
SP_BDM_exlarge_rv_y,0.0143,1.0000


,ethereum_gt_rv_x,SP_BDM_rv_x
ethereum_gt_rv_y,1.0,0.0908
SP_BDM_rv_y,0.0,1.0000


,ethereum_gt_rv_x,ether_rv_x
ethereum_gt_rv_y,1.0000,0.7336
ether_rv_y,0.0004,1.0000


In [ ]:
for file in filelist:
  for coin in coinlist:
    title = [file[64:-7]+"_rv",coin[54:-7]+"_rv"]
    df_sentiment = pd.read_csv(file)
    df_sentiment.columns = ['date','index']
    df_sentiment['change'] = df_sentiment['index'].pct_change()
    df_sentiment['sen_vol'] = df_sentiment['change'].rolling(30).std()
    df_sentiment['sen_rv'] = df_sentiment['sen_vol'].pct_change()
    df_coin = pd.read_csv(coin)
    df_coin['return'] = df_coin['price'].pct_change()
    df_coin['coin_vol'] = df_coin['return'].rolling(30).std()
    df_coin['coin_rv'] = df_coin['coin_vol'].pct_change()
    df_new = df_sentiment.merge(df_coin, left_on = "date", right_on = "Date", how="left")
    df_new.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_new.dropna(inplace=True)

    variables = ['coin_rv','sen_rv']
    grangers_causation_matrix(title, df_new, variables)

,4.s&p_twitter_rv_x,bitcoin_rv_x
4.s&p_twitter_rv_y,1.0000,0.5453
bitcoin_rv_y,0.0217,1.0000


,4.s&p_twitter_rv_x,SP_BDM_exlarge_rv_x
4.s&p_twitter_rv_y,1.0000,0.0023
SP_BDM_exlarge_rv_y,0.0004,1.0000


,4.s&p_twitter_rv_x,SP_BDM_rv_x
4.s&p_twitter_rv_y,1.0000,0.0214
SP_BDM_rv_y,0.0032,1.0000


,4.s&p_twitter_rv_x,ether_rv_x
4.s&p_twitter_rv_y,1.0000,0.3396
ether_rv_y,0.0053,1.0000


,10.news_senti_rv_x,bitcoin_rv_x
10.news_senti_rv_y,1.0000,0.3938
bitcoin_rv_y,0.1973,1.0000


...

,twitter_btccrash_pos_rv_x,ether_rv_x
twitter_btccrash_pos_rv_y,1.0000,0.7197
ether_rv_y,0.0041,1.0000


,twitter_btccrash_neg_rv_x,bitcoin_rv_x
twitter_btccrash_neg_rv_y,1.0000,0.6844
bitcoin_rv_y,0.4779,1.0000


,twitter_btccrash_neg_rv_x,SP_BDM_exlarge_rv_x
twitter_btccrash_neg_rv_y,1.0000,0.5984
SP_BDM_exlarge_rv_y,0.2815,1.0000


,twitter_btccrash_neg_rv_x,SP_BDM_rv_x
twitter_btccrash_neg_rv_y,1.0000,0.6525
SP_BDM_rv_y,0.3052,1.0000


,twitter_btccrash_neg_rv_x,ether_rv_x
twitter_btccrash_neg_rv_y,1.0000,0.6304
ether_rv_y,0.2955,1.0000
